In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

pd.set_option('max_columns',100)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,10)

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
y_train = df_train['SalePrice']
X_train = df_train.drop(['SalePrice','Id'], axis=1)
X_test = df_test.drop(['Id'], axis=1)


In [40]:
predict_vars1 = ['LotArea','GrLivArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','FullBath',\
                 'HalfBath','BedroomAbvGr','KitchenAbvGr','GarageCars','MoSold','YrSold',\
                 'Neighborhood','KitchenQual','ExterQual']
predict_vars2 = ['LotArea','GrLivArea','YearBuilt','FullBath',\
                 'BedroomAbvGr','KitchenAbvGr','GarageCars']
predict_vars3 = ['LotArea','GrLivArea','OverallQual','YearBuilt','FullBath','BedroomAbvGr']

In [67]:
df1 = df_train[predict_vars1]  
df2 = df_train[predict_vars2]
df3 = df_train[predict_vars3]

In [72]:
df1 = pd.get_dummies(df1, drop_first=True, dummy_na=True)
#consider switching MoSold, Qual & Cond variables to categorical?

In [74]:
df1.head()
# print(df2.head())
# print(df3.head())


,LotArea,GrLivArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,GarageCars,MoSold,YrSold,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Neighborhood_nan,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,KitchenQual_nan,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterQual_nan
0,8450,1710,7,5,2003,2003,2,1,3,1,2,2,2008,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,9600,1262,6,8,1976,1976,2,0,3,1,2,5,2007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
2,11250,1786,7,5,2001,2002,2,1,3,1,2,9,2008,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,9550,1717,7,5,1915,1970,1,0,3,1,3,2,2006,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,14260,2198,8,5,2000,2000,2,1,4,1,3,12,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [75]:
#Split Kaggle (training) data set into training & test sets (80/20)

seed = 0
x1_train,x1_test,y1_train,y1_test = train_test_split(df1,y_train,test_size=0.20,random_state=seed)
x2_train,x2_test,y2_train,y2_test = train_test_split(df2,y_train,test_size=0.20,random_state=seed)
x3_train,x3_test,y3_train,y3_test = train_test_split(df3,y_train,test_size=0.20,random_state=seed)


In [77]:
#LINEAR REGRESSION:  df1   

linear = linear_model.LinearRegression()
linear.fit(x1_train, y1_train)
linear.score(x1_train, y1_train)
print("Coefficient: \n", linear.coef_)
print("Intercept: \n", linear.intercept_)
print("R² Value: \n", linear.score(x1_train, y1_train))
predicted = linear.predict(x1_test)    **** BEST result so far (R² = 0.86)

Coefficient: 
 [  7.04062570e-01   6.62162037e+01   1.04550605e+04   5.60849136e+03
   4.78352977e+02   7.03251959e+01  -1.45944369e+03  -8.40723711e+03
  -4.87458884e+03  -2.29059821e+04   1.00904692e+04  -7.01915271e+02
  -4.20798227e+02  -1.67779094e+04  -1.45532032e+04   1.07719823e+04
   2.05739317e+04   1.41740075e+04   3.43235120e+04  -9.75443843e+02
   2.56241855e+03  -1.31498123e+03  -1.15308553e+04   6.74835571e+03
   9.67388439e+03  -8.36379812e+02   5.65724375e+03   6.66154996e+04
   4.54386814e+04  -1.38247739e+03  -1.31961719e+03   8.56250094e+03
   6.39613287e+03   1.81308815e+04   5.79946846e+04   2.03488434e+04
   2.70571333e+04   9.09494702e-12  -3.43845375e+04  -3.64519519e+04
  -3.88863706e+04   9.09494702e-13  -3.56810160e+04  -3.87202643e+04
  -3.79420550e+04   0.00000000e+00]
Intercept: 
 -170115.27521
R² Value: 
 0.858630421196


In [79]:
#LINEAR REGRESSION:  df2

linear = linear_model.LinearRegression()
linear.fit(x2_train, y2_train)
linear.score(x2_train, y2_train)
print("Coefficient: \n", linear.coef_)
print("Intercept: \n", linear.intercept_)
print("R² Value: \n", linear.score(x2_train, y2_train))
predicted = linear.predict(x2_test)

Coefficient: 
 [  7.12089008e-01   9.93841651e+01   6.90789717e+02   2.39271610e+02
  -1.51265795e+04  -4.19445360e+04   2.02266998e+04]
Intercept: 
 -1287942.75136
R² Value: 
 0.746485277235


In [80]:
#LINEAR REGRESSION:  df3

linear = linear_model.LinearRegression()
linear.fit(x3_train, y3_train)
linear.score(x3_train, y3_train)
print("Coefficient: \n", linear.coef_)
print("Intercept: \n", linear.intercept_)
print("R² Value: \n", linear.score(x3_train, y3_train))
predicted = linear.predict(x3_test)

Coefficient: 
 [  9.27113959e-01   7.92816724e+01   2.26139645e+04   5.57274819e+02
  -7.52387131e+03  -1.18160499e+04]
Intercept: 
 -1139544.91885
R² Value: 
 0.778138772315


In [78]:
#DECISION TREES:  df1

from sklearn import tree

decision = tree.DecisionTreeRegressor()
decision.fit(x1_train, y1_train)
print("Accuracy: \n", decision.score(x1_test, y1_test))

Accuracy: 
 0.805620847792


In [81]:
#DECISION TREES:  df2

decision = tree.DecisionTreeRegressor()
decision.fit(x2_train, y2_train)
print("Accuracy: \n", decision.score(x2_test, y2_test))

Accuracy: 
 0.693951940578


In [82]:
#DECISION TREES:  df3

decision = tree.DecisionTreeRegressor()
decision.fit(x3_train, y3_train)
print("Accuracy: \n", decision.score(x3_test, y3_test))

Accuracy: 
 0.779114845358
